In [ ]:


from typing import List, Callable, Tuple ,Set , Dict
import numpy as np
import faiss
from sklearn.preprocessing import normalize

print(1)
methods_list={}

In [ ]:
# Generate sample data
num_entries = 1000
num_queries = 10
dims = [4, 8, 12]  # Example dimensions for different embedding types

# Create sample database entries
database = [
    [np.random.random((dims[i],)).astype('float32') for i in range(len(dims))]
    for _ in range(num_entries)
]

# Create sample queries
queries = [
    [np.random.random((dims[i],)).astype('float32') for i in range(len(dims))]
    for _ in range(num_queries)
]


In [ ]:

# Type aliases for clarity
EmbeddingsList = List[List[np.ndarray]]  # List of lists of embeddings
SearchFunction = Callable[[List[np.ndarray]], Tuple[np.ndarray, np.ndarray]]

def simple_concatenation_ann(data: EmbeddingsList) -> SearchFunction:
    """
    Simple concatenation method. Concatenates embeddings as-is.

    Args:
        data: List where each entry is a list of embeddings (one per embedding type)

    Returns:
        search_function: Function that takes query embeddings and returns distances and indices
    """
    # Validate and process data
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")

    # Get dimensions and validate consistency
    num_embedding_types = len(data[0])
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    total_dim = sum(dims)

    # Concatenate all data
    concatenated_data = np.hstack([
        np.vstack([entry[i] for entry in data])
        for i in range(num_embedding_types)
    ]).astype('float32')

    # Create and train index
    index = faiss.IndexHNSWFlat(total_dim, 16)
    index.hnsw.efConstruction = 100
    index.hnsw.efSearch = 64
    index.add(concatenated_data)

    def search(query_embeddings: List[np.ndarray], k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")

        concatenated_query = np.hstack(query_embeddings).astype('float32').reshape((1,-1))
        distances, indices = index.search(concatenated_query, k)
        return distances, indices

    return search

methods_list["Simple Concatenation"] =simple_concatenation_ann

In [ ]:
# Type aliases
EmbeddingsList = List[List[np.ndarray]]
SearchFunction = Callable[[List[np.ndarray]], Tuple[np.ndarray, np.ndarray]]

def multi_index_ann(data: EmbeddingsList) -> SearchFunction:
    """
    Creates separate indices for each embedding type and combines their results,
    calculating distances to all embeddings for each candidate.
    
    Args:
        data: List where each entry is a list of embeddings (one per embedding type)
    
    Returns:
        search_function: Function that takes query embeddings and returns combined results
    """
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")
    
    num_embedding_types = len(data[0])
    entry_count = len(data)
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    
    # Create separate indices and store embeddings for each type
    indices = []
    embeddings_by_type = []
    
    for i in range(num_embedding_types):
        embeddings = np.vstack([entry[i] for entry in data]).astype('float32')
        embeddings_by_type.append(embeddings)
        
        index = faiss.IndexHNSWFlat(dims[i], 16)
        index.hnsw.efConstruction = 100
        index.hnsw.efSearch = 64
        index.add(embeddings)
        indices.append(index)
    
    def search(query_embeddings: List[np.ndarray], k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")
        
        # Search in each index separately to get initial candidate set
        all_candidates: Set[int] = set()
        
        for i, (query, index) in enumerate(zip(query_embeddings, indices)):
            query_array = query.reshape(1, -1).astype('float32')
            _, indices_found = index.search(query_array, k)
            all_candidates.update(indices_found[0])
        
        # Calculate distances for all candidates across all embedding types
        candidate_scores = []
        
        for idx in all_candidates:
            total_distance = 0
            for i, query in enumerate(query_embeddings):
                # Reshape for broadcasting
                query_vector = query.reshape(1, -1)
                db_vector = embeddings_by_type[i][idx].reshape(1, -1)
                
                # Calculate L2 distance
                distance = np.linalg.norm(query_vector - db_vector)
                total_distance += distance
            
            candidate_scores.append((idx, total_distance))
        
        # Sort by total distance and get top k
        candidate_scores.sort(key=lambda x: x[1])
        final_indices = [idx for idx, _ in candidate_scores[:k]]
        final_distances = [dist for _, dist in candidate_scores[:k]]
        
        return (np.array(final_distances).reshape(1, -1), 
                np.array(final_indices).reshape(1, -1))
    
    return search


methods_list["separate indexing"] =multi_index_ann

In [ ]:
def normalized_scaled_ann(data: EmbeddingsList) -> SearchFunction:
    """
    Normalized and scaled concatenation method. Normalizes each embedding type
    and scales based on dimension size.
    
    Args:
        data: List where each entry is a list of embeddings (one per embedding type)
    
    Returns:
        search_function: Function that takes query embeddings and returns distances and indices
    """
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")
    
    
    num_embedding_types = len(data[0])
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    total_dim = sum(dims)
    
    # Calculate scaling factors based on dimensions
    max_dim = max(dims)
    scaling_factors = [np.sqrt(max_dim / dim) for dim in dims]
    
    # Normalize and scale each embedding type, then concatenate
    #do Z-score normalization.
    # Find the maximum length of the embeddings in each column
    numberOfEmbeddings =len(data[0])
    max_lengths = [len(data[0][i]) for i in range(numberOfEmbeddings)]
    
    # Initialize lists to store sums and counts for each column
    sums = [np.zeros(mlengths) for mlengths in max_lengths]
    counts = len(data)
    
    # First pass: Calculate sums and counts for each index of embeddings in each column
    for row in data:
        for i in range(numberOfEmbeddings):
            sums[i]+=row[i]
    
    
    # Calculate means for each index of embeddings in each column
    means = [su /counts for su in sums]
    
    # Initialize lists to store squared differences for each column
    squared_diffs = [np.zeros(mlengths) for mlengths in max_lengths]
    
    # Second pass: Calculate squared differences from the mean for each index of embeddings in each column
    for row in data:
        for i in range(numberOfEmbeddings):
            squared_diffs[i]+= np.power(row[i]- means[i],2)
    
    # Calculate standard deviations for each index of embeddings in each column
    stds = [np.sqrt(squared_diff / counts) for squared_diff in squared_diffs]
    
    # to avoid division by zero
    for std in stds:
        std+= (std ==0 ) # mask is 1 where std == 0 
    # Normalize the data
    normalized_data = []
    for row in data:
        normalized_row = []
        for mean ,std,ro in zip(means ,stds,row ):
            normalized_row.append((ro-mean)/std)
        normalized_data.append(normalized_row)

    concatenated_data = np.hstack([
        np.vstack([entry[i] for entry in normalized_data]) * scaling_factors[i]
        for i in range(num_embedding_types)
    ]).astype('float32')
    
    # Create and train index
    index = faiss.IndexHNSWFlat(total_dim, 16)
    index.hnsw.efConstruction = 100
    index.hnsw.efSearch = 64
    index.add(concatenated_data)
    
    def search(query_embeddings: List[np.ndarray], k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")
        
        # Apply the same normalization and scaling to query embeddings
        normalized_scaled_query = np.hstack([
            ((query_embeddings[i]-means[i])/stds[i]) * scaling_factors[i]
            for i in range(num_embedding_types)
        ]).astype('float32').reshape(1,-1)
        
        distances, indices = index.search(normalized_scaled_query, k)
        #todo efficiently find real distances before normalizing
        distances = [ ]
        unnormalized_query =np.hstack([i for i in query_embeddings])
        for idx in indices[0]:
            unnormalized_vector =np.hstack([i for i in data[idx]])
            distances.append(np.linalg.norm(unnormalized_vector-unnormalized_query))
        distances = np.array([distances]).reshape((1,-1))
        return distances, indices
    
    return search

methods_list["Normalized and Scaled"] =normalized_scaled_ann

In [ ]:
from dataclasses import dataclass
@dataclass
class RobustSearchConfig:
    min_embeddings_required: int = 1  # Minimum embeddings needed for a match
    use_rank_fusion: bool = True      # Whether to use rank-based fusion
    outlier_threshold: float = 2.0    # Z-score threshold for outlier detection
    rank_weights: List[float] = None  # Weights for different embedding types
    
def robust_multi_index_ann(data: EmbeddingsList) -> SearchFunction:
    """
    Creates a robust ANN search that can handle unreliable embeddings.
    
    Args:
        data: List where each entry is a list of embeddings (one per embedding type)
    
    Returns:
        search_function: Function that takes query embeddings and returns robust results
    """
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")
    
    num_embedding_types = len(data[0])
    entry_count = len(data)
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    
    # Create indices and store embeddings
    indices = []
    embeddings_by_type = []
    
    # Calculate mean and std for each embedding type for normalization
    means = []
    stds = []
    
    for i in range(num_embedding_types):
        embeddings = np.vstack([entry[i] for entry in data]).astype('float32')
        embeddings_by_type.append(embeddings)
        
        # Store mean and std for normalization
        means.append(np.mean(embeddings, axis=0))
        stds.append(np.std(embeddings, axis=0))
        
        index = faiss.IndexHNSWFlat(dims[i], 16)
        index.hnsw.efConstruction = 100
        index.hnsw.efSearch = 64
        index.add(embeddings)
        indices.append(index)
    
    def reciprocal_rank_fusion(rankings: List[List[int]], k: int = 60) -> Dict[int, float]:
        """Combine multiple rankings using reciprocal rank fusion"""
        fused_scores = {}
        for rank_list in rankings:
            for rank, idx in enumerate(rank_list):
                if idx not in fused_scores:
                    fused_scores[idx] = 0
                fused_scores[idx] += 1.0 / (rank + k)
        return fused_scores
    
    def calculate_normalized_distances(query: np.ndarray, embeddings: np.ndarray, 
                                      mean: np.ndarray, std: np.ndarray) -> np.ndarray:
        """Calculate normalized distances accounting for distribution"""
        normalized_query = (query - mean) / std
        normalized_embeddings = (embeddings - mean) / std
        return np.linalg.norm(normalized_embeddings - normalized_query, axis=1)
    
    def search(query_embeddings: List[np.ndarray], k: int = 5, 
               config: RobustSearchConfig = RobustSearchConfig()) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")
        
        # Initialize storage for rankings and distances
        all_rankings: List[List[int]] = []
        all_distances: List[Dict[int, float]] = []
        all_candidates: Set[int] = set()
        
        # Search in each index and store results
        for i, (query, index) in enumerate(zip(query_embeddings, indices)):
            query_array = query.reshape(1, -1).astype('float32')
            distances, indices_found = index.search(query_array, k)
            
            # Store rankings and distances
            all_rankings.append(indices_found[0].tolist())
            all_candidates.update(indices_found[0])
            
            # Calculate normalized distances for all candidates
            normalized_distances = calculate_normalized_distances(
                query, embeddings_by_type[i], means[i], stds[i])
            all_distances.append({idx: dist for idx, dist in enumerate(normalized_distances)})
        
        # Combine results using various robust methods
        candidate_scores = []
        
        if config.use_rank_fusion:
            # Use rank fusion for initial scoring
            fused_scores = reciprocal_rank_fusion(all_rankings)
            initial_candidates = set(fused_scores.keys())
        else:
            initial_candidates = all_candidates
        
        weights = config.rank_weights or [1.0] * num_embedding_types
        
        for idx in initial_candidates:
            embedding_scores = []
            valid_embeddings = 0
            
            for i in range(num_embedding_types):
                distance = all_distances[i][idx]
                
                # Check if this embedding is an outlier
                if abs(distance - np.mean(list(all_distances[i].values()))) < \
                   config.outlier_threshold * np.std(list(all_distances[i].values())):
                    embedding_scores.append(distance * weights[i])
                    valid_embeddings += 1
            
            # Only consider if we have enough valid embeddings
            if valid_embeddings >= config.min_embeddings_required:
                # Use median of scores to be robust to outliers
                median_score = np.median(embedding_scores)
                candidate_scores.append((idx, median_score))
        
        # Sort and get top k
        candidate_scores.sort(key=lambda x: x[1])
        final_indices = [idx for idx, _ in candidate_scores[:k]]
        final_distances = [dist for _, dist in candidate_scores[:k]]
        
        return (np.array(final_distances).reshape(1, -1), 
                np.array(final_indices).reshape(1, -1))
    
    return search
methods_list["robust multi index"] =robust_multi_index_ann

In [ ]:
def dimension_reduction_ann(data: EmbeddingsList, target_dim: int = 128) -> SearchFunction:
    """
    Dimension reduction method. Reduces total dimensions to a target size.
    
    Args:
        data: List where each entry is a list of embeddings (one per embedding type)
        target_dim: Target dimension for the reduced space (default 128)
    
    Returns:
        search_function: Function that takes query embeddings and returns distances and indices
    """
    # Validate and process data
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")
    

    # Get dimensions and validate consistency
    num_embedding_types = len(data[0])
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    total_original_dim = sum(dims)
    
    target_dim =  min(target_dim,round(total_original_dim*3 / 4 ))
    
    # Concatenate all data
    concatenated_data = np.hstack([
        np.vstack([entry[i] for entry in data])
        for i in range(num_embedding_types)
    ]).astype('float32')
    
    # Create PCA reducer
    reducer = faiss.PCAMatrix(total_original_dim, target_dim)
    
    # Train reducer on the data
    reducer.train(concatenated_data)
    
    # Apply reduction to the data
    reduced_data = reducer.apply_py(concatenated_data)
    
    # Create and train index on reduced data
    index = faiss.IndexHNSWFlat(target_dim, 16)
    index.hnsw.efConstruction = 100
    index.hnsw.efSearch = 64
    index.add(reduced_data)
    
    def search(query_embeddings: List[np.ndarray], k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")
        
        # Concatenate query embeddings
        concatenated_query = np.hstack(query_embeddings).astype('float32').reshape(1, -1)
        
        # Reduce query dimensions
        reduced_query = reducer.apply_py(concatenated_query)
        
        # Perform search
        distances, indices = index.search(reduced_query, k)
        return distances, indices
    
    return search

methods_list["Dimension Reduction"] = dimension_reduction_ann

In [ ]:
def normal_lsh_ann(data: EmbeddingsList, num_planes: int = 256) -> SearchFunction:
    """
    Normal LSH method. Applies LSH to concatenated embeddings.
    
    Args:
        data: List where each entry is a list of embeddings (one per embedding type)
        num_planes: Number of hyperplanes for LSH
    
    Returns:
        search_function: Function that takes query embeddings and returns distances and indices
    """
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")
    
    num_embedding_types = len(data[0])
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    total_dim = sum(dims)
    
    # Concatenate all data
    concatenated_data = np.hstack([
        np.vstack([entry[i] for entry in data])
        for i in range(num_embedding_types)
    ]).astype('float32')
    
    # Create and train LSH index
    index = faiss.IndexLSH(total_dim, num_planes)
    index.train(concatenated_data)
    index.add(concatenated_data)
    
    def search(query_embeddings: List[np.ndarray], k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")
        
        concatenated_query = np.hstack(query_embeddings).astype('float32').reshape(1, -1)
        distances, indices = index.search(concatenated_query, k)
        return distances, indices
    
    return search

def split_lsh_ann(data: EmbeddingsList, num_planes: int = 256) -> SearchFunction:
    """
    Split LSH method. Applies LSH separately to each embedding type.
    
    Args:
        data: List where each entry is a list of embeddings (one per embedding type)
        num_planes: Total number of hyperplanes to be distributed among embedding types
    
    Returns:
        search_function: Function that takes query embeddings and returns distances and indices
    """
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")
    
    num_embedding_types = len(data[0])
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    total_dim = sum(dims)
    
    # Allocate planes proportionally to dimension sizes
    planes_per_type = [max(1, int(num_planes * dim / total_dim)) for dim in dims]
    
    # Adjust to ensure total equals num_planes
    while sum(planes_per_type) != num_planes:
        if sum(planes_per_type) < num_planes:
            idx = dims.index(max(dims))
            planes_per_type[idx] += 1
        else:
            idx = dims.index(min(dims))
            if planes_per_type[idx] > 1:
                planes_per_type[idx] -= 1
    
    # Create separate LSH indexes for each embedding type
    indexes = []
    for i, dim in enumerate(dims):
        index = faiss.IndexLSH(total_dim, planes_per_type[i])
        
        # Create masked training data
        masked_data = np.zeros((len(data), total_dim), dtype='float32')
        start_idx = sum(dims[:i])
        masked_data[:, start_idx:start_idx+dims[i]] = np.vstack([entry[i] for entry in data])
        
        index.train(masked_data)
        index.add(masked_data)
        indexes.append(index)
    
    def search(query_embeddings: List[np.ndarray], k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")
        
        concatenated_query = np.hstack(query_embeddings).astype('float32').reshape(1, -1)
        
        # Combine results from all indexes
        all_distances = []
        all_indices = []
        
        for i, index in enumerate(indexes):
            distances, indices = index.search(concatenated_query, k)
            all_distances.append(distances)
            all_indices.append(indices)
        
        # Merge and sort results
        merged_distances = np.hstack(all_distances)
        merged_indices = np.hstack(all_indices)
        
        # Sort by distance and take top k
        sorted_indices = np.argsort(merged_distances[0])[:k]
        final_distances = merged_distances[0][sorted_indices].reshape(1, -1)
        final_indices = merged_indices[0][sorted_indices].reshape(1, -1)
        
        return final_distances, final_indices
    
    return search

methods_list["Normal LSH"] = normal_lsh_ann
methods_list["Split LSH"] = split_lsh_ann

In [ ]:
def tolerant_ann(data: EmbeddingsList, subset_size: int = None) -> SearchFunction:
    """
    Outlier-tolerant ANN method. Creates multiple indexes using different subsets of embeddings.
    Finds matches that are close in most, but not necessarily all, embedding spaces.
    
    Args:
        data: List where each entry is a list of embeddings (one per embedding type)
        subset_size: Number of embedding types to combine in each subset (default: num_types - 1)
    
    Returns:
        search_function: Function that takes query embeddings and returns distances and indices
    """
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")
    
    num_embedding_types = len(data[0])
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    
    if subset_size is None:
        subset_size = max(1, num_embedding_types - 1)
    
    if subset_size > num_embedding_types:
        raise ValueError(f"subset_size ({subset_size}) cannot be larger than number of embedding types ({num_embedding_types})")
    
    # Create all possible combinations of embedding types
    from itertools import combinations
    embedding_combinations = list(combinations(range(num_embedding_types), subset_size))
    
    # Create an index for each combination
    indexes = []
    for combo in embedding_combinations:
        # Calculate total dimension for this combination
        combo_dim = sum(dims[i] for i in combo)
        
        # Concatenate only the embeddings in this combination
        combo_data = np.hstack([
            np.vstack([entry[i] for entry in data])
            for i in combo
        ]).astype('float32')
        
        # Create and train index
        index = faiss.IndexHNSWFlat(combo_dim, 16)
        index.hnsw.efConstruction = 100
        index.hnsw.efSearch = 64
        index.add(combo_data)
        
        indexes.append((combo, index))
    
    def search(query_embeddings: List[np.ndarray], k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")
        
        # Search in each index and aggregate results
        all_results = {}  # Dictionary to store index -> score mapping
        
        for combo, index in indexes:
            # Concatenate only the query embeddings for this combination
            combo_query = np.hstack([query_embeddings[i] for i in combo]).astype('float32').reshape(1, -1)
            
            # Search in this index
            distances, indices = index.search(combo_query, k)
            
            # Accumulate scores for each found index
            for idx, dist in zip(indices[0], distances[0]):
                if idx not in all_results:
                    all_results[idx] = []
                all_results[idx].append(dist)
        
        # Aggregate scores using a method that rewards consistency across subsets
        final_scores = []
        for idx, distances in all_results.items():
            # Use average of best scores as the final score
            best_scores = sorted(distances)[:max(1, len(indexes) - num_embedding_types + subset_size)]
            score = sum(best_scores) / len(best_scores)
            final_scores.append((score, idx))
        
        # Sort by score and get top k
        final_scores.sort()
        top_k = final_scores[:k]
        
        # Format results to match expected output
        result_distances = np.array([[score for score, _ in top_k]])
        result_indices = np.array([[idx for _, idx in top_k]])
        
        return result_distances, result_indices
    
    return search

methods_list["Tolerant ANN"] = tolerant_ann

In [ ]:
def capped_distance_ann(data: EmbeddingsList, cap_distance: float = 0.7) -> SearchFunction:
    """
    ANN method using capped distances for each embedding type.
    Limits the impact of outlier embeddings by capping their contribution to the total distance.
    
    Args:
        data: List where each entry is a list of embeddings (one per embedding type)
        cap_distance: Maximum distance contribution from each embedding type
    
    Returns:
        search_function: Function that takes query embeddings and returns distances and indices
    """
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")
    
    num_embedding_types = len(data[0])
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    total_dim = sum(dims)
    
    # Concatenate all data
    concatenated_data = np.hstack([
        np.vstack([entry[i] for entry in data])
        for i in range(num_embedding_types)
    ]).astype('float32')
    
    # Create a custom index with capped distance metric
    class CappedDistanceIndex:
        def __init__(self, data, dims, cap):
            self.data = data
            self.dims = dims
            self.cap = cap
            self.dim_starts = [0] + list(np.cumsum(dims[:-1]))
    
        def search(self, query, k):
            # Compute distances for each embedding type separately
            n = len(self.data)
            all_distances = np.zeros((len(query), n), dtype=np.float32)
            
            for i, start in enumerate(self.dim_starts):
                end = start + self.dims[i]
                # Compute squared L2 distance for this embedding type
                delta = self.data[:, start:end] - query[:, start:end]
                distances = np.sum(delta * delta, axis=1)
                # Cap the distances
                np.minimum(distances, self.cap * self.cap, out=distances)
                all_distances += distances
            
            # Use argpartition to efficiently find top k
            ind = np.argpartition(all_distances[0], k)[:k]
            # Sort the top k
            ind_sorted = ind[np.argsort(all_distances[0][ind])]
            distances_sorted = all_distances[0][ind_sorted]
            
            return np.sqrt(distances_sorted.reshape(1, -1)), ind_sorted.reshape(1, -1)
    
    # Create the index
    index = CappedDistanceIndex(concatenated_data, dims, cap_distance)
    
    def search(query_embeddings: List[np.ndarray], k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")
        
        concatenated_query = np.hstack(query_embeddings).astype('float32').reshape(1, -1)
        distances, indices = index.search(concatenated_query, k)
        return distances, indices
    
    return search

methods_list["Capped Distance"] = capped_distance_ann

In [ ]:
def emphasis_close_ann(data: EmbeddingsList, boost_threshold: float = 0.3) -> SearchFunction:
    """
    ANN method that emphasizes close matches in any embedding type.
    If any embedding type shows a very close match, it reduces the impact of other distances.
    
    Args:
        data: List where each entry is a list of embeddings (one per embedding type)
        boost_threshold: Distance threshold below which a match is considered "close"
                        and triggers emphasis behavior
    
    Returns:
        search_function: Function that takes query embeddings and returns distances and indices
    """
    if not data or not all(data):
        raise ValueError("Data must be non-empty and all entries must have embeddings")
    
    num_embedding_types = len(data[0])
    dims = [data[0][i].shape[-1] for i in range(num_embedding_types)]
    
    # Normalize embeddings for consistent distance scaling
    normalized_data = []
    for i in range(num_embedding_types):
        embeddings = np.vstack([entry[i] for entry in data])
        norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
        normalized_data.append(embeddings / norms)
    
    # Create separate indexes for each embedding type for efficient search
    indexes = []
    for i, emb in enumerate(normalized_data):
        index = faiss.IndexHNSWFlat(dims[i], 16)
        index.hnsw.efConstruction = 100
        index.hnsw.efSearch = 64
        index.add(emb.astype('float32'))
        indexes.append(index)
    
    def search(query_embeddings: List[np.ndarray], k: int = 5) -> Tuple[np.ndarray, np.ndarray]:
        if len(query_embeddings) != num_embedding_types:
            raise ValueError(f"Expected {num_embedding_types} embedding types, got {len(query_embeddings)}")
        
        # Normalize query embeddings
        normalized_queries = []
        for query_emb in query_embeddings:
            norm = np.linalg.norm(query_emb)
            normalized_queries.append((query_emb / norm).astype('float32').reshape(1, -1))
        
        # Initial wide search in each embedding space
        expanded_k = min(k * 3, len(data))  # Search for more candidates initially
        all_candidates = set()
        per_type_results = []
        
        for i, (index, query) in enumerate(zip(indexes, normalized_queries)):
            distances, indices = index.search(query, expanded_k)
            all_candidates.update(indices[0])
            per_type_results.append((distances[0], indices[0]))
        
        # Calculate final scores for all candidates
        final_scores = []
        for idx in all_candidates:
            type_distances = []
            close_matches = 0
            
            # Collect distances for this candidate across all embedding types
            for type_idx, (distances, indices) in enumerate(per_type_results):
                if idx in indices:
                    dist = distances[np.where(indices == idx)[0][0]]
                    if dist < boost_threshold:
                        close_matches += 1
                    type_distances.append(dist)
                else:
                    # If this candidate wasn't in top-k for this embedding type,
                    # use a default high distance
                    type_distances.append(1.0)
            
            # Calculate final score
            # The more close matches, the more we discount other distances
            discount_factor = 1.0 / (close_matches + 1)
            non_close_distances = sorted([d for d in type_distances if d >= boost_threshold])
            
            if close_matches > 0:
                # Use the best close distance, plus discounted sum of other distances
                final_score = min(type_distances) + sum(non_close_distances) * discount_factor
            else:
                # If no close matches, use regular average
                final_score = sum(type_distances) / len(type_distances)
            
            final_scores.append((final_score, idx))
        
        # Sort and get top k
        final_scores.sort()
        top_k = final_scores[:k]
        
        # Format results
        result_distances = np.array([[score for score, _ in top_k]])
        result_indices = np.array([[idx for _, idx in top_k]])
        
        return result_distances, result_indices
    
    return search

methods_list["Emphasis Close"] = emphasis_close_ann

In [ ]:
 # simple 
# Test all methods
for method_name,method in methods_list.items():
    print(f"\nTesting {method_name}")
    if True:
        search_fn = method(database)
        distances, indices = search_fn(queries[0])
        
        print(f"Top 5 results for first query:")
        for i in range(1):
            print(f"  Index: {indices[0][i]}, Distance: {distances[0][i]:.4f}")


In [ ]:
import numpy as np
from collections import defaultdict
import heapq
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw, ImageFont

def display_images_with_labels(image1, image2, image3):
    # Create a figure and axis
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Display images
    axes[0].imshow(image1)
    axes[0].set_title("Query image")
    axes[0].axis('off')
    
    axes[1].imshow(image2)
    axes[1].set_title("Agreed similar")
    axes[1].axis('off')
    
    axes[2].imshow(image3)
    axes[2].set_title("Proposed alternative similar")
    axes[2].axis('off')
    
    # Display the plot
    plt.show()



def compare_similarity_methods(methods_list, database, queries, top_k=5, result_size=5, display_score_limit = 2.5, dispplay_picture_funcation=display_images_with_labels , display_picture = None , queries_images = None):
    results = {}
    disagreement_scores = defaultdict(list)

    # Run all methods on all queries
    for method_name, method in methods_list.items():
        search_fn = method(database)
        method_results = []
        
        for query in queries:
            distances, indices = search_fn(query)
            method_results.append((distances[0][:top_k], indices[0][:top_k]))
        
        results[method_name] = method_results
        
    def spot_rank(spot: int): # gets unsigned int only . dont find out plz 
        rank = 1/(1+spot)
        return  rank

    # Compare methods and calculate disagreement scores
    for query_idx, query in enumerate(queries):
        rank_results = defaultdict(lambda : 0)
        for method_name in methods_list.keys():
            for spot , entry in enumerate( results[method_name][query_idx][1]):
                rank_results[entry] += spot_rank(spot)
        max_entry = max(rank_results , key=lambda r: rank_results[r])
        max_rank = rank_results[max_entry]
        
        for method_name in methods_list.keys():
            current_method_results = results[method_name][query_idx][1]
            
            entry_id = current_method_results[0]
            # Calculate disagreement score
            disagreement_score = max_rank -rank_results[entry_id]
            
            if disagreement_score > 0:  # Only consider cases where the method ranks an entry higher than others
                heapq.heappush(disagreement_scores[method_name], 
                               (-disagreement_score, query_idx, max_entry, entry_id))
                
                # Keep only the top 'result_size' disagreements
                if len(disagreement_scores[method_name]) > result_size:
                    heapq.heappop(disagreement_scores[method_name])

    # Print results
    for method_name in methods_list.keys():
        print(f"\nTop {result_size} disagreements for {method_name}:")
        max_diff = 0 
        qi , ari , pei = 0,0,0
        for score, query_id, agreed_result, proposed_entry in sorted(disagreement_scores[method_name], reverse=True):
            if -score > max_diff:
                max_diff = - score
                qi , ari , pei = query_id, agreed_result, proposed_entry
            print(f"  Score difference: {-score:.2f}")
            print(f"  Query ID: {query_id}")
            print(f"  Agreed result ID: {agreed_result}")
            print(f"  Proposed entry ID: {proposed_entry}")
            print()
        if dispplay_picture_funcation is not None and display_picture is not None and queries_images is not None:
            if display_score_limit < max_diff:
                dispplay_picture_funcation(queries_images[qi], display_picture[ari], display_picture[pei])


In [ ]:
# create embeddings for images 
embedding_functions = {}

In [ ]:
import numpy as np
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.cluster import KMeans
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms.functional import to_tensor
from skimage.feature import graycomatrix, graycoprops
from torchvision.models import resnet50, ResNet50_Weights , VGG16_Weights
import torchvision.models as PTmodels
from torchvision.models.detection import  FasterRCNN_ResNet50_FPN_Weights

In [ ]:
def cnn_embedding(img, model_name='vgg16'):
    """
    Generate CNN features using a pre-trained model.
    
    Args:
    image_path (str): Path to the input image
    model_name (str): Name of the pre-trained model to use
    
    Returns:
    np.array: Feature vector
    """
    # Load pre-trained model
    if model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
    elif model_name == 'vgg16':
        model = PTmodels.vgg16(weights=PTmodels.VGG16_Weights.IMAGENET1K_V1)
    else:
        raise ValueError("Unsupported model name")
    
    model = torch.nn.Sequential(*list(model.children())[:-1])
    model.eval()
    
    # Prepare image
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    #img = Image.open(image_path).convert('RGB')
    # Ensure input is PIL Image
    if not isinstance(img, Image.Image):
        img = transforms.ToPILImage()(img)
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    
    # Generate embedding
    with torch.no_grad():
        embedding = model(batch_t)
    
    return embedding.numpy().flatten()

def color_palette_embedding(img, n_colors=5):
    """
    Generate color palette embedding using K-means clustering.
    
    Args:
    image_path (str): Path to the input image
    n_colors (int): Number of colors in the palette
    
    Returns:
    np.array: Color palette embedding
    """
    # Load and prepare image
    #img = Image.open(image_path).convert('RGB')
    #if not isinstance(img, np.ndarray):
    if len(img.shape )>2:
        img = np.array(img).reshape(-1, 3)
    
    # Perform K-means clustering
    kmeans = KMeans(n_clusters=n_colors, random_state=42)
    kmeans.fit(img)
    
    # Get color palette and normalize
    palette = kmeans.cluster_centers_.astype(int)
    normalized_palette = palette / 255.0
    
    return normalized_palette.flatten()

def object_composition_embedding(img, threshold=0.5):
    """
    Generate object composition embedding using a pre-trained Faster R-CNN model.
    
    Args:
    image_path (str): Path to the input image
    threshold (float): Confidence threshold for object detection
    
    Returns:
    dict: Object composition (class labels and their counts)
    """
    const_number_of_class_in_fasterrcnn_resnet50_fpn  =91
    # Load pre-trained model
    #model = fasterrcnn_resnet50_fpn(pretrained=True)
    model = PTmodels.detection.fasterrcnn_resnet50_fpn(weights=PTmodels.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
    model.eval()
    
    # Prepare image
    #img = Image.open(image_path).convert('RGB')
    img_tensor = to_tensor(img).unsqueeze(0)
    
    # Perform object detection
    with torch.no_grad():
        prediction = model(img_tensor)
    
    # Process predictions
    labels = prediction[0]['labels'].numpy()
    scores = prediction[0]['scores'].numpy()
    
    # Filter predictions based on threshold
    valid_preds = scores > threshold
    detected_labels = labels[valid_preds]
    
    # Count occurrences of each label
    unique, counts = np.unique(detected_labels, return_counts=True)
    composition = [0 for _ in range(const_number_of_class_in_fasterrcnn_resnet50_fpn)]
    for u,c in zip(unique, counts):
        composition[u] =c
    
    return np.array(composition)

def texture_embedding(img, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4]):
    """
    Generate texture embedding using Gray Level Co-occurrence Matrix (GLCM).
    
    Args:
    image_path (str): Path to the input image
    distances (list): List of pixel pair distance offsets
    angles (list): List of pixel pair angles in radians
    
    Returns:
    np.array: Texture features
    """
    # Load and prepare image
    if not isinstance(img, np.ndarray):
        img = img.convert('L')  # Convert to grayscale
        img = np.array(img)
    if len(img.shape) >2:
        img = np.array(img).reshape(-1, 3)
    if np.issubdtype(img.dtype, np.floating):
        img = (img * 255).astype(np.uint8)
    # Compute GLCM
    glcm = graycomatrix(img, distances=distances, angles=angles, 
                        levels=256, symmetric=True, normed=True)
    
    # Compute GLCM properties
    contrast = graycoprops(glcm, 'contrast')
    dissimilarity = graycoprops(glcm, 'dissimilarity')
    homogeneity = graycoprops(glcm, 'homogeneity')
    energy = graycoprops(glcm, 'energy')
    correlation = graycoprops(glcm, 'correlation')
    
    # Combine features
    features = np.hstack([contrast, dissimilarity, homogeneity, energy, correlation])
    
    return features.flatten()

embedding_functions["cnn_embedding"] = cnn_embedding
embedding_functions["color_palette_embedding"] = color_palette_embedding
embedding_functions["object_composition_embedding"] = object_composition_embedding
embedding_functions["texture_embedding"] = texture_embedding

In [ ]:
def orb_embedding(img, n_features=100): # image_path
    """
    Generate ORB (Oriented FAST and Rotated BRIEF) features for an image.
    
    Args:
    image_path (str): Path to the input image
    n_features (int): Maximum number of features to extract
    
    Returns:
    np.array: Fixed-length feature vector combining descriptor information
    """
    import cv2
    import numpy as np
    
    # Read image in grayscale
    #img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if not isinstance(img, np.ndarray):
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)

    if img is None:
        raise ValueError("Could not load image")
    
    # Initialize ORB detector
    orb = cv2.ORB_create(nfeatures=n_features)
    
    # Detect keypoints and compute descriptors
    keypoints, descriptors = orb.detectAndCompute(img, None)
    
    if descriptors is None:
        # Return zero vector if no features found
        return np.zeros(n_features * 8, dtype=np.float32)
    
    # Ensure we have a fixed-length output
    if len(keypoints) < n_features:
        # Pad with zeros if we found fewer features than requested
        padding = np.zeros((n_features - len(keypoints), 32), dtype=np.uint8)
        descriptors = np.vstack((descriptors, padding))
    elif len(keypoints) > n_features:
        # Take only the first n_features if we found more
        descriptors = descriptors[:n_features]
    
    # Convert binary descriptors to float and flatten
    descriptors = descriptors.astype(np.float32)
    
    # Normalize the descriptors
    descriptors = descriptors / 255.0
    
    # Flatten the descriptor array
    flattened = descriptors.flatten()
    
    return flattened


embedding_functions["orb_embedding"] = orb_embedding


In [ ]:
import numpy as np
from PIL import Image
import random

import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR100
from sklearn.cluster import KMeans
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms.functional import to_tensor
from skimage.feature import graycomatrix, graycoprops
from skimage.measure import shannon_entropy
from skimage.filters import sobel
from skimage.transform import hough_line, hough_line_peaks
import cv2
from transformers import CLIPProcessor, CLIPModel
import matplotlib.pyplot as plt


In [ ]:
# Helper embedding functions
def resnet18_embedding(img, layer='avgpool'):
    """
    Compute ResNet18 embedding for an image
    
    Args:
    img (PIL.Image or torch.Tensor): Input image
    layer (str): Layer to extract embedding from
    
    Returns:
    np.ndarray: Flattened embedding vector
    """
    # Load pretrained ResNet18
    model = PTmodels.resnet18(weights=PTmodels.ResNet18_Weights.IMAGENET1K_V1)
    model.eval()
    
    # Preprocessing
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    #if not isinstance(img, np.ndarray):
        # Ensure input is PIL Image
    if not isinstance(img, Image.Image):
        img = transforms.ToPILImage()(img)
            
            # Preprocess image
    input_tensor = preprocess(img).unsqueeze(0)
    
    # Extract features
    with torch.no_grad():
        if layer == 'avgpool':
            # Use average pooling layer
            features = model.avgpool(model.layer4(model.layer3(model.layer2(model.layer1(model.conv1(input_tensor))))))
        elif layer == 'fc':
            # Use final fully connected layer
            features = model.fc(model.avgpool(model.layer4(model.layer3(model.layer2(model.layer1(model.conv1(input_tensor))))).flatten(1)))
    
    return features.squeeze().numpy().flatten()

def simple_color_histogram_embedding(img, bins=32):
    """
    Compute a simple color histogram embedding
    
    Args:
    img (PIL.Image or torch.Tensor): Input image
    bins (int): Number of bins for color histogram
    
    Returns:
    np.ndarray: Flattened color histogram
    """
    if not isinstance(img, np.ndarray):
        if not isinstance(img, Image.Image):
            img = transforms.ToPILImage()(img)
        img = np.array(img)
    
    # Compute histogram for each color channel
    hist_r = np.histogram(img[:,:,0], bins=bins, range=[0,256])[0]
    hist_g = np.histogram(img[:,:,1], bins=bins, range=[0,256])[0]
    hist_b = np.histogram(img[:,:,2], bins=bins, range=[0,256])[0]
    
    # Normalize and concatenate
    hist = np.concatenate([
        hist_r / np.sum(hist_r),
        hist_g / np.sum(hist_g),
        hist_b / np.sum(hist_b)
    ])
    
    return hist

embedding_functions["resnet18_embedding"]=resnet18_embedding
embedding_functions["simple_color_histogram_embedding"]=simple_color_histogram_embedding

In [ ]:

def composition_rules_embedding(img):#image_path):
    """
    Generate embedding based on basic photography composition rules.
    
    Args:
    image_path (str): Path to the input image
    
    Returns:
    np.array: Composition features
    """
    #img = np.array(Image.open(image_path).convert('RGB'))
    if not isinstance(img, np.ndarray):
        img = np.array(img.convert('RGB'))
    h, w = img.shape[:2]
    
    # Rule of thirds points
    third_h = h // 3
    third_w = w // 3
    
    # Calculate feature importance at rule of thirds intersections
    thirds_points = [
        img[third_h, third_w],
        img[third_h, 2*third_w],
        img[2*third_h, third_w],
        img[2*third_h, 2*third_w]
    ]
    
    # Center weight
    center_region = img[h//3:2*h//3, w//3:2*w//3]
    center_weight = np.mean(center_region)
    
    # Convert to features
    thirds_features = np.mean(thirds_points, axis=1)
    
    return np.concatenate([thirds_features.flatten(), [center_weight]])

def scene_complexity_embedding(img):#image_path):
    """
    Generate embedding based on image complexity metrics.
    
    Args:
    image_path (str): Path to the input image
    
    Returns:
    np.array: Complexity features
    """
    #img = np.array(Image.open(image_path).convert('L'))
    if not isinstance(img, np.ndarray):
        img = np.array(Image.convert('L'))
    
    # Calculate entropy
    entropy = shannon_entropy(img)
    
    # Calculate frequency domain features
    f_transform = np.fft.fft2(img)
    f_spectrum = np.abs(np.fft.fftshift(f_transform))
    freq_energy = np.sum(f_spectrum)
    
    # Calculate number of unique intensity values
    unique_intensities = len(np.unique(img))
    
    # Calculate local variance
    local_var = np.std(img)
    
    return np.array([entropy, freq_energy, unique_intensities, local_var])

def semantic_concept_embedding(image):#image_path):
    """
    Generate embedding based on high-level semantic concepts using CLIP.
    
    Args:
    image_path (str): Path to the input image
    
    Returns:
    np.array: Semantic concept features
    """
    # Load CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    
    # Prepare image
    #image = Image.open(image_path)
    inputs = processor(images=image, return_tensors="pt", padding=True)
    
    # Generate embedding
    with torch.no_grad():
        image_features = model.get_image_features(**inputs)
    
    return image_features.numpy()[0]

embedding_functions["composition_rules_embedding"] = composition_rules_embedding
embedding_functions["scene_complexity_embedding"] = scene_complexity_embedding
embedding_functions["semantic_concept_embedding"] = semantic_concept_embedding

In [ ]:
def get_sample_images(n_images=25):
    """Get n_images unique images from CIFAR100 dataset"""
    # Load CIFAR100 dataset
    dataset = CIFAR100(root='./data', train=True, download=True)
    
    # Randomly select n_images unique indices
    selected_indices = random.sample(range(len(dataset)), n_images)
    
    # Get the images
    images = []
    for idx in selected_indices:
        img, _ = dataset[idx]
        images.append(img)
    
    return images

def display_image_grid(images, n_rows=5, n_cols=5):
    """Display images in a grid"""
    plt.figure(figsize=(15, 15))
    for idx, image in enumerate(images):
        if idx >= n_rows * n_cols:
            break
        plt.subplot(n_rows, n_cols, idx + 1)
        plt.imshow(image)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

def apply_rotation_changes(image):
    """Apply different levels of rotation"""
    transforms_list = [
        transforms.RandomRotation(degrees=(0, 0)),  # No rotation
        transforms.RandomRotation(degrees=(2, 2)),  # Slight rotation
        transforms.RandomRotation(degrees=(90, 90)), # Moderate rotation
        transforms.RandomRotation(degrees=(60, 60))  # Severe rotation
    ]
    return [transform(image) for transform in transforms_list]

def apply_brightness_changes(image):
    """Apply different levels of brightness adjustment"""
    transforms_list = [
        transforms.ColorJitter(brightness=0),  # Original
        transforms.ColorJitter(brightness=0.2),  # Slight brightness change
        transforms.ColorJitter(brightness=0.5),  # Moderate brightness change
        transforms.ColorJitter(brightness=0.8)   # Severe brightness change
    ]
    return [transform(image) for transform in transforms_list]

def apply_noise_changes(image):
    """Apply different levels of Gaussian noise"""
    def add_gaussian_noise(img, std):
        img_array = np.array(img)
        noise = np.random.normal(0, std, img_array.shape)
        noisy_img = np.clip(img_array + noise, 0, 255).astype(np.uint8)
        return Image.fromarray(noisy_img)
    
    return [
        image,  # Original
        add_gaussian_noise(image, 10),  # Light noise
        add_gaussian_noise(image, 25),  # Medium noise
        add_gaussian_noise(image, 50)   # Heavy noise
    ]

def apply_blur_changes(image):
    """Apply different levels of Gaussian blur"""
    transforms_list = [
        transforms.GaussianBlur(kernel_size=1, sigma=0.1),  # No blur
        transforms.GaussianBlur(kernel_size=3, sigma=1.0),  # Light blur
        transforms.GaussianBlur(kernel_size=5, sigma=2.0),  # Medium blur
        transforms.GaussianBlur(kernel_size=7, sigma=3.0)   # Heavy blur
    ]
    return [transform(image) for transform in transforms_list]

def create_transformation_dataset(images):
    """Create dataset with all transformation levels for each image"""
    all_transformed_images = {}
    
    transformation_functions = {"rotation": apply_rotation_changes,
                                "brightness":apply_brightness_changes,
                                "noise":apply_noise_changes,
                                "blur":apply_blur_changes,
                                }
    for transformation_name ,transform_func in transformation_functions.items():
        result = []
        for idx, image in enumerate(images):
            # Select transformation function based on image index
            transformed_images = transform_func(image)
            result.extend(transformed_images)
        all_transformed_images[transformation_name] = result
    
    return all_transformed_images



In [ ]:
# Example usage

# Get 25 sample images
original_images = get_sample_images(25)

# Display original images in 5x5 grid
display_image_grid(original_images[:9])

# Create transformed dataset
transformed_dataset = create_transformation_dataset(original_images)

# Display first 25 transformed images (you can modify this to view different subsets)
display_image_grid(transformed_dataset["noise"][:8])

#transformed_dataset

In [ ]:
import torch
import numpy as np
from scipy.spatial.distance import cdist

def compute_embedding_scores(embeddings):
    """
    Compute scores for transformed images based on their embedding distances
    
    Args:
    embeddings (torch.Tensor or np.ndarray): 100 image embeddings
    
    Returns:
    list: Scores for each set of 4 transformed images
    """
    # Ensure embeddings are numpy array
    if torch.is_tensor(embeddings):
        embeddings = embeddings.numpy()
    
    # Number of sets (25 original images * 4 transformation groups)
    num_sets = len(embeddings) // 4 # 25 
    scores = []
    
    for set_idx in range(num_sets):
        # Select the base (original) image embedding
        base_index = set_idx * 4
        
        
        # Select the current set of 4 embeddings
        current_set_embeddings = embeddings[base_index:base_index+4]
        base_embedding = current_set_embeddings[0]
        
        # Compute distances between base embedding and all 100 embeddings
        distances = cdist([base_embedding], embeddings, metric='cosine')[0]
        
        # Compute ranks for each transformation
        ranks = []
        sorted_indices = np.argsort(distances)
        # Find the rank of the original images in this set
        for i in range(base_index,base_index+4):
            ranks.append(np.where(sorted_indices == i)[0][0])
        
        # Advanced scoring method
        # Exponential penalty for higher ranks with non-linear decay
        score = (
            4 * np.exp(-ranks[0]/10) +  # Original image
            3 * np.exp(-ranks[1]/20) +  # Lightly transformed
            2 * np.exp(-ranks[2]/30) +  # Moderately transformed
            1 * np.exp(-ranks[3]/50)    # Heavily transformed
        )
        
        scores.append(score)
    
    return scores


In [ ]:
import time
def measure_embedding_times(image, embedding_functions: dict):
    for name, func in embedding_functions.items():
        print("for funcation :", name)
        start_time = time.time()
        func(np.array(image))
        end_time = time.time()
        print("the time is:", end_time - start_time)

measure_embedding_times(original_images[0] , embedding_functions )

In [ ]:
import numpy as np
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F

def comprehensive_embedding_analysis(
    image_transformations, 
    embedding_functions, 
    verbose=True
):
    """
    Analyze embedding performance across different transformation types and embedding methods
    
    Args:
    image_transformations (dict): {transform_name: [100 images]}
    embedding_functions (dict): {embedding_name: callable embedding function}
    verbose (bool): Whether to print detailed results
    
    Returns:
    dict: Comprehensive analysis results
    """
    # Final results storage
    analysis_results = {}
    
    # Iterate through each embedding method
    for embed_name, embed_func in embedding_functions.items():
        # Store results for this embedding method
        embed_results = {}
        
        # Iterate through each transformation type
        for transform_name, images in image_transformations.items():
            # Convert images to numpy array of embeddings
            try:
                embeddings = np.array([
                    embed_func(img) for img in images
                ])
            except Exception as e:
                print(f"Error embedding {transform_name} with {embed_name}: {e}")
                continue
            
            # Compute embedding scores
            try:
                scores = compute_embedding_scores(embeddings)
                
                # Compute statistics
                mean_score = np.mean(scores)
                std_score = np.std(scores)
                
                # Store results
                embed_results[transform_name] = {
                    'scores': scores,
                    'mean_score': mean_score,
                    'std_score': std_score
                }
            except Exception as e:
                print(f"Error computing scores for {transform_name} with {embed_name}: {e}")
        
        # Store embedding method results
        analysis_results[embed_name] = embed_results
    
    # Verbose printing
    if verbose:
        print("\n===== Embedding Analysis Results =====")
        for embed_name, embed_results in analysis_results.items():
            print(f"\n{embed_name} Embedding Analysis:")
            for transform_name, result in embed_results.items():
                print(f"  {transform_name}:")
                print(f"    Mean Score: {result['mean_score']:.4f}")
                print(f"    Std Dev:    {result['std_score']:.4f}")
    
    return analysis_results

def example_usage(embedding_functions,image_transformations):
    """
    Example of how to use comprehensive embedding analysis
    
    Args:
    image_transformations (dict): Dictionary of image transformations
    """
    # Perform analysis
    results = comprehensive_embedding_analysis(
        image_transformations, 
        embedding_functions
    )
    
    return results

# Assuming previous code for image transformations exists
if __name__ == "__main__":
    

    output = {'cnn_embedding': {'rotation': {'mean_score': 9.279951468969303, 'std_score': 0.5228738273544796}, 'brightness': {'mean_score': 9.619479076729254, 'std_score': 0.06287745571148899}, 'noise': {'mean_score': 9.129639231631808, 'std_score': 0.4850666314145876}, 'blur': {'mean_score': 9.652409504800124, 'std_score': 0.06516211199870199}}, 'color_palette_embedding': {'rotation': {'mean_score': 7.0150218676586, 'std_score': 1.176705144422394}, 'brightness': {'mean_score': 8.712775297566337, 'std_score': 1.0907352053398691}, 'noise': {'mean_score': 7.7002460606126, 'std_score': 1.5819974433642798}, 'blur': {'mean_score': 7.981503805928394, 'std_score': 1.3263521120918198}}, 'object_composition_embedding': {'rotation': {'mean_score': 3.086884624261154, 'std_score': 2.8514783444668628}, 'brightness': {'mean_score': 3.112322959102469, 'std_score': 2.8862478155235403}, 'noise': {'mean_score': 2.718949958295171, 'std_score': 2.2813804200135217}, 'blur': {'mean_score': 2.811589758475318, 'std_score': 2.3897654189664537}}, 'texture_embedding': {'rotation': {'mean_score': 7.017257251536493, 'std_score': 0.968019802142182}, 'brightness': {'mean_score': 9.589879223913742, 'std_score': 0.0825360389506324}, 'noise': {'mean_score': 8.14887967219288, 'std_score': 0.943276245624005}, 'blur': {'mean_score': 8.04787621205756, 'std_score': 1.6008730912006686}}, 'orb_embedding': {'rotation': {'mean_score': 2.135176705559059, 'std_score': 2.1090008976254984}, 'brightness': {'mean_score': 2.135176705559059, 'std_score': 2.1090008976254984}, 'noise': {'mean_score': 2.135176705559059, 'std_score': 2.1090008976254984}, 'blur': {'mean_score': 2.135176705559059, 'std_score': 2.1090008976254984}}, 'resnet18_embedding': {'rotation': {'mean_score': 9.545184057802482, 'std_score': 0.24819643985265794}, 'brightness': {'mean_score': 8.484506255217056, 'std_score': 0.9845392175554086}, 'noise': {'mean_score': 8.60332950904624, 'std_score': 0.5746430131022944}, 'blur': {'mean_score': 9.298168605217189, 'std_score': 0.5691158240400043}}, 'simple_color_histogram_embedding': {'rotation': {'mean_score': 9.422409803329568, 'std_score': 0.17054289277673787}, 'brightness': {'mean_score': 8.144029551066467, 'std_score': 1.268061064579965}, 'noise': {'mean_score': 9.474449473405622, 'std_score': 0.2165506856022929}, 'blur': {'mean_score': 9.64832012123764, 'std_score': 0.04131197998927866}}, 'composition_rules_embedding': {'rotation': {'mean_score': 7.940926598618239, 'std_score': 0.7697549086658149}, 'brightness': {'mean_score': 9.573509926856365, 'std_score': 0.07961568110678785}, 'noise': {'mean_score': 9.504592599260107, 'std_score': 0.18506826100322477}, 'blur': {'mean_score': 9.424750032396684, 'std_score': 0.3663873556989816}}, 'scene_complexity_embedding': {'rotation': {'mean_score': 7.599703335612207, 'std_score': 1.0356749158247425}, 'brightness': {'mean_score': 8.236728590414288, 'std_score': 0.8341092305914172}, 'noise': {'mean_score': 6.8927261455636595, 'std_score': 0.9460073418551256}, 'blur': {'mean_score': 5.223397028017563, 'std_score': 1.0596380864286603}}, 'semantic_concept_embedding': {'rotation': {'mean_score': 9.430091119057082, 'std_score': 0.2928156456935988}, 'brightness': {'mean_score': 9.599496867957804, 'std_score': 0.11692920739396973}, 'noise': {'mean_score': 8.572574984770254, 'std_score': 1.1324747931842993}, 'blur': {'mean_score': 9.202430635899812, 'std_score': 0.45761894948833565}}}
    #output = None
    if output is None: # this takes 1 hour
        results = example_usage(embedding_functions, transformed_dataset)
    else:
        print("\n===== Embedding Analysis Results =====")
        for embed_name, embed_results in output.items():
            print(f"\n{embed_name} Embedding Analysis:")
            for transform_name, result in embed_results.items():
                print(f"  {transform_name}:")
                print(f"    Mean Score: {result['mean_score']:.4f}")
                print(f"    Std Dev:    {result['std_score']:.4f}")


In [ ]:
good= ["simple_color_histogram_embedding" ,"composition_rules_embedding", "color_palette_embedding","resnet18_embedding","texture_embedding" , "cnn_embedding","semantic_concept_embedding"]
good_embedding_functions = {}
for key ,item in embedding_functions.items():
    if key in good:
        good_embedding_functions[key] = item

In [ ]:
def measure_embedding_times(image_list: Image.Image, embedding_functions: dict):
    for name, func in embedding_functions.items():
        print("for funcation :", name)
        start_time = time.time()
        for image in image_list[:5]:
            func(np.array(image))
        end_time = time.time()
        print("the time is:", end_time - start_time)

measure_embedding_times(original_images , good_embedding_functions )

In [ ]:
too_slow = [ "cnn_embedding", "semantic_concept_embedding"]
for key in too_slow:
    good_embedding_functions.pop(key)

In [ ]:
###### start heavy testing 

In [ ]:
def get_all_embedings(image , embedding_functions = good_embedding_functions):
    return [ ef(image) for ef in embedding_functions.values()]

In [ ]:
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cosine
import os
import requests
import zipfile
from tqdm import tqdm
import shutil

In [ ]:
def download_tiny_imagenet(base_dir):
    """
    Downloads and extracts Tiny ImageNet dataset
    """
    os.makedirs(base_dir, exist_ok=True)
    url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    zip_path = os.path.join(base_dir, "tiny-imagenet-200.zip")

    if not os.path.exists(zip_path):
        print("Downloading Tiny ImageNet...")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))

        with open(zip_path, 'wb') as f:
            for data in tqdm(response.iter_content(chunk_size=1024), total=total_size // 1024):
                f.write(data)

    if not os.path.exists(os.path.join(base_dir, "tiny-imagenet-200")):
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(base_dir)


def get_similar_image_pairs_tiny(num_pairs,pair_size = 2,  base_dir="./dataset", similarity_threshold=0.7, image_size=64):
    """
    Returns pairs of similar Tiny ImageNet images where pairs are from the same class,
    with additional similarity check for better matching.

    Args:
        num_pairs (int): Number of pairs to return
        base_dir (str): Directory to store/load dataset
        similarity_threshold (float): Minimum similarity for paired images
        image_size (int): Size to resize images to (Tiny ImageNet images are 64x64)

    Returns:
        numpy array: Selected images array of shape (num_pairs*2, image_size, image_size, 3)
        list: Class names for each pair
    """
    # Download and extract dataset if needed
    download_tiny_imagenet(base_dir)

    # Create feature extractor for similarity checks
    feature_extractor = tf.keras.applications.MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=(image_size, image_size, 3),
        pooling='avg'
    )

    # Path to training images
    train_dir = os.path.join(base_dir, "tiny-imagenet-200", "train")

    # Dictionary to store images by class
    class_images = {}
    class_features = {}
    selected_images = []
    selected_class_names = []
    used_classes = set()

    def load_and_preprocess_image(image_path):
        """Load and preprocess image for feature extraction"""
        img = tf.io.read_file(image_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (image_size, image_size))
        img = tf.cast(img, tf.float32)
        return img, tf.keras.applications.mobilenet_v2.preprocess_input(img)

    # Load class data
    print("Loading dataset...")
    class_dirs = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]

    class_dirs = random.sample(class_dirs, num_pairs)

    for class_dir in tqdm(class_dirs):
        class_path = os.path.join(train_dir, class_dir)
        image_files = [f for f in os.listdir(os.path.join(class_path, "images"))
                       if f.endswith('.JPEG')][:min(500 , 10+  pair_size) ]  # Take up to 10 images per class

        if len(image_files) >= pair_size:  # Only include classes with at least 2 images
            class_images[class_dir] = []
            class_features[class_dir] = []

            for img_file in image_files:
                img_path = os.path.join(class_path, "images", img_file)
                original_img, processed_img = load_and_preprocess_image(img_path)
                features = feature_extractor.predict(tf.expand_dims(processed_img, 0), verbose=0)

                class_images[class_dir].append(original_img)
                class_features[class_dir].append(features.flatten())

    print("Finding similar pairs...")
    """
    for _ in tqdm(range(num_pairs)):
        # Find an unused class
        available_classes = [c for c in class_images.keys()
                             if c not in used_classes and len(class_images[c]) >= pair_size]

        if not available_classes:
            raise ValueError(f"Not enough unique classes with sufficient images. "
                             f"Only found {len(selected_images) // pair_size} pairs.")

        # Pick a random unused class
        chosen_class = np.random.choice(available_classes)
        """
    for chosen_class in class_images.keys():
        if len(class_images[chosen_class]) <= pair_size:
            raise ValueError(f"Not enough unique classes with sufficient images. "
                             f"Only found {len(selected_images) // pair_size} pairs.")
        used_classes.add(chosen_class)

        # Get features for this class
        class_feat = class_features[chosen_class]
        class_imgs = class_images[chosen_class]

        # Pick first image randomly
        first_idx = np.random.randint(len(class_imgs))
        first_features = class_feat[first_idx]

        # Find most similar image
        similarities = [1 - cosine(first_features, feat) for feat in class_feat]
        similarities[first_idx] = -1  # Exclude the same image

        selected_class_names.append(chosen_class)
        selected_images.extend([class_imgs[first_idx].numpy()])
        #second_idx = np.argmax(similarities)
        pair_inxs = np.argpartition(similarities, pair_size - 1 )[ -(pair_size-1):]
        # Add the pair to our selection

        for second_idx in pair_inxs:
            selected_images.extend([ class_imgs[second_idx].numpy()])


    return np.array(selected_images), selected_class_names


def visualize_tiny_imagenet_pairs(images, class_names ,pair_size = 2  ):
    """
    Helper function to display the image pairs side by side with their class names

    Args:
        images (numpy array): Array of images to display
        class_names (list): List of class names for each pair
    """

    num_pairs = len(images) // pair_size
    fig, axes = plt.subplots(num_pairs, pair_size, figsize=(10, 5 * num_pairs))

    if num_pairs == 1:
        axes = axes.reshape(1, -1)

    for i in range(num_pairs):
        for j in range(pair_size):
            idx = i * pair_size + j
            axes[i, j].imshow(images[idx].astype('uint8'))
            axes[i, j].axis('off')
            if j == 0:
                axes[i, j].set_title(f'Class: {class_names[i]}')

    plt.tight_layout()
    plt.show()

# Get 3 pairs of similar images (6 images total)
images, class_names = get_similar_image_pairs_tiny(2 ,pair_size =4)

# Visualize the pairs with their class names
visualize_tiny_imagenet_pairs(images, class_names , pair_size =4 )

In [ ]:
def display_images_with_labels_nparray(image1, image2, image3):
    # Create a figure and axis
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Display images
    axes[0].imshow(image1.astype('uint8'))
    axes[0].set_title("Query image")
    axes[0].axis('off')
    
    axes[1].imshow(image2.astype('uint8'))
    axes[1].set_title("Agreed similar")
    axes[1].axis('off')
    
    axes[2].imshow(image3.astype('uint8'))
    axes[2].set_title("Proposed alternative similar")
    axes[2].axis('off')
    
    # Display the plot
    plt.show()

In [ ]:
numer_of_images_per_class = 8 # todo make this very big

In [ ]:
pair_similar_images, _ = get_similar_image_pairs_tiny(200 ,pair_size = numer_of_images_per_class)

In [ ]:
start_time = time.time()
pair_sample_embd = [  get_all_embedings(i) for i in pair_similar_images]
print( time.time() - start_time)

In [ ]:
import time
import numpy as np
from typing import Dict, Callable, Tuple, List
import pandas as pd


def evaluate_ann_search(
    embedded_images: List[np.ndarray],
    ann_methods: Dict[str, Callable],
    num_searches: int,
    pair_size: int = 2,
    K: int = 20,
    display = True
) -> pd.DataFrame:
    """
    Evaluates the performance of different ANN search functions on the provided embedded images.

    Args:
        embedded_images (List[np.ndarray]): List of embedded images, where each element is a
                                            numpy array representing the embeddings for one image.
        ann_methods (Dict[str, Callable]): Dictionary of ANN search functions, where the
                                           keys are the method names and the values are the
                                           search functions.
        num_searches (int): Number of searches to perform for each ANN method.
        pair_size (int, optional): Number of images per pair. Defaults to 2.
        K (int, optional): Number of nearest neighbors to retrieve. Defaults to 20.
        display (bool, optional): Whether to display additional information. Defaults to False.

    Returns:
        pd.DataFrame: Comprehensive performance metrics for each ANN search method
    """
    # Define comprehensive columns for performance metrics
    columns = [
        'Method Name',
        'Index Build Time',
        'Average Search Time',
        'Accuracy@K',
        'Recall@K',
        'Mean Average Precision@K',
        'Diversity Score'
    ]

    # Initialize results storage
    results = []

    # Ensure number of searches doesn't exceed available data
    num_searches = min(num_searches, len(embedded_images) // pair_size)

    # Randomly select search pairs
    pair_indices = np.random.choice(
        range(len(embedded_images) // pair_size),
        size=num_searches,
        replace=False
    ) * pair_size

    # Iterate through each ANN method
    for method_name, ANN_func in ann_methods.items():
        print(f"Evaluating method: {method_name}")

        # Performance tracking variables
        search_times = []
        accuracies = []
        recalls = []
        map_scores = []
        diversity_scores = []

        # Index building time
        start_time = time.time()
        search_func = ANN_func(embedded_images)
        index_build_time = time.time() - start_time

        # Perform searches
        for indice in pair_indices:
            # Run the search
            start_time = time.time()
            distances, indices = search_func(embedded_images[indice], K)
            search_time = time.time() - start_time

            search_times.append(search_time)

            query_pairs = set([int(indice)+ i for i in range(1,pair_size )])
            set_inces = set(indices[:K].tolist()[0])
            matches = len(set_inces & query_pairs)

            # Calculate various metrics
            # Accuracy: Proportion of correct matches in top K results
            accuracies.append(matches / K )

            # Recall: Proportion of ground truth neighbors found
            recalls.append(matches / (pair_size -1))


            # Mean Average Precision
            precisions = []
            for i in range(1, K+1):
                precision_at_k = len(set(indices[:i].tolist()[0]) & query_pairs) / i
                precisions.append(precision_at_k)
            map_scores.append(np.mean(precisions))


            # Diversity Score: Measures the variety of retrieved results
            # Calculate variance of distances as a simple diversity metric
            diversity_scores.append(np.std(distances))

        # Compile method-level metrics
        method_results = {
            'Method Name': method_name,
            'Index Build Time': index_build_time,
            'Average Search Time': np.mean(search_times),
            'Accuracy@K': np.mean(accuracies),
            'Recall@K': np.mean(recalls),
            'Mean Average Precision@K': np.mean(map_scores),
            'Diversity Score': np.mean(diversity_scores)
        }

        results.append(method_results)

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)

    # Optional display
    if display:
        print(results_df)

    return results_df




In [ ]:
evaluate_ann_search(pair_sample_embd , methods_list , num_searches= 100 , pair_size= numer_of_images_per_class,K= 20 ,display= False )